In [39]:
import json
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import JSON, DOUBLE, VARCHAR
import os,glob
PROVINCES = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/geojson/provinces.json'
CENSUS_DIVISIONS = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/geojson/censusDivisions.json'
CENSUS_SUBDIVISIONS = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/geojson/subdivisions.json'
DISSEMINATION_AREAS = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/geojson/disseminationAreas.json'


In [40]:
def depth(l):
    if isinstance(l, list):
        return 1 + max(depth(item) for item in l)
    else:
        return 0

In [41]:
# Local Connection
ssl_args = {'ssl_ca': "/etc/ssl/cert.pem"}

conn_string = 'mysql+pymysql://' + "root" + ':' + "Chuckie2020!" + '@' + "localhost" + '/' + "canada_vis" 

engine = create_engine(conn_string, connect_args=ssl_args)

upload_sql = []

In [4]:
# Connecting to Planet Scale
ssl_args = {'ssl_ca': "/etc/ssl/cert.pem"}

conn_string = 'mysql+pymysql://' + os.getenv("USERNAME") + ':' + os.getenv("PASSWORD") + '@' + os.getenv("HOST") + '/' + os.getenv("DATABASE") 

engine = create_engine(conn_string, connect_args=ssl_args)

In [42]:
with open(PROVINCES) as provincesjson:
    provinces = json.load(provincesjson)
    
for feature in provinces["features"]:
    feature["properties"]["geo_name"] = feature["properties"].pop('PRENAME')
    feature["properties"]['CDUID'] = "0"
    feature["geometry"]["type"] = "MultiPolygon"
    feature['geo_level'] = "province"
    if (depth(feature['geometry']['coordinates'])) == 3:
        feature['geometry']['coordinates'] = [feature['geometry']['coordinates']] 


for feature in provinces["features"]:
    new_data = {}
    new_data["geo_level"] = feature['geo_level']
    new_data["dguid"] = feature['properties']['DGUID']
    new_data["pruid"] = feature['properties']['PRUID']
    new_data["geo_name"] = feature['properties']['geo_name']
    new_data["land_area"] = feature['properties']['LANDAREA']
    new_data["coordinates"] = feature["geometry"]["coordinates"]
    upload_sql.append(new_data)

In [43]:
with open(CENSUS_DIVISIONS) as census_divisions_json:
    census_divisions = json.load(census_divisions_json)

    print(census_divisions['features'][0]['properties'])

for feature in census_divisions["features"]:
    feature["properties"]["geo_name"] = feature["properties"].pop('CDNAME')
    feature["properties"]['CDUID'] = "0"
    feature["geometry"]["type"] = "MultiPolygon"
    feature['geo_level'] = "census division"
    if (depth(feature['geometry']['coordinates'])) == 3:
        feature['geometry']['coordinates'] = [feature['geometry']['coordinates']] 

for feature in census_divisions["features"]:
    new_data = {}
    new_data["geo_level"] = feature['geo_level']
    new_data["dguid"] = feature['properties']['DGUID']
    new_data["pruid"] = feature['properties']['PRUID']
    new_data["geo_name"] = feature['properties']['geo_name']
    new_data["land_area"] = feature['properties']['LANDAREA']
    new_data["coordinates"] = feature["geometry"]["coordinates"]
    upload_sql.append(new_data)

{'CDUID': '1001', 'DGUID': '2021A00031001', 'CDNAME': 'Division No.  1', 'CDTYPE': 'CDR', 'LANDAREA': 9104.5799, 'PRUID': '10'}


In [44]:
with open(CENSUS_SUBDIVISIONS) as census_subdivisions_json:
    census_subdivisions = json.load(census_subdivisions_json)

    print(census_subdivisions['features'][0]['properties'])

for feature in census_subdivisions["features"]:
    feature["properties"]["geo_name"] = feature["properties"].pop('CSDNAME')
    feature["properties"]['CDUID'] = "0"
    feature["geometry"]["type"] = "MultiPolygon"
    feature['geo_level'] = "census sub division"
    if (depth(feature['geometry']['coordinates'])) == 3:
        feature['geometry']['coordinates'] = [feature['geometry']['coordinates']] 

for feature in census_subdivisions["features"]:
    new_data = {}
    new_data["geo_level"] = feature['geo_level']
    new_data["dguid"] = feature['properties']['DGUID']
    new_data["pruid"] = feature['properties']['PRUID']
    new_data["geo_name"] = feature['properties']['geo_name']
    new_data["land_area"] = feature['properties']['LANDAREA']
    new_data["coordinates"] = feature["geometry"]["coordinates"]
    upload_sql.append(new_data)

{'CSDUID': '1001101', 'DGUID': '2021A00051001101', 'CSDNAME': 'Division No.  1, Subd. V', 'CSDTYPE': 'SNO', 'LANDAREA': 870.8928, 'PRUID': '10'}


In [45]:
with open(DISSEMINATION_AREAS) as dissemination_areas_json:
    disseminationAreas = json.load(dissemination_areas_json)

    print(disseminationAreas['features'][0]['properties'])

for feature in disseminationAreas["features"]:
    feature["properties"]["geo_name"] = feature["properties"].pop('DAUID')
    feature["properties"]['CDUID'] = "0"
    feature["geometry"]["type"] = "MultiPolygon"
    feature['geo_level'] = "dissemination area"
    if (depth(feature['geometry']['coordinates'])) == 3:
        feature['geometry']['coordinates'] = [feature['geometry']['coordinates']] 

for feature in disseminationAreas["features"]:
    new_data = {}
    new_data["geo_level"] = feature['geo_level']
    new_data["dguid"] = feature['properties']['DGUID']
    new_data["pruid"] = feature['properties']['PRUID']
    new_data["geo_name"] = feature['properties']['geo_name']
    new_data["land_area"] = feature['properties']['LANDAREA']
    new_data["coordinates"] = feature["geometry"]["coordinates"]
    upload_sql.append(new_data)

{'DAUID': '10010165', 'DGUID': '2021S051210010165', 'LANDAREA': 0.3817, 'PRUID': '10'}


PRUID, DGUID, PRNAME, PRENAME, PRFNAME, PREABBR, PPRFABBR, LANDAREA
pop
PRNAME
PRFNAME
PREABBR
PRFABBR

In [46]:
data_types_dict = {'geo_level' : VARCHAR(191),
                   'dguid' : VARCHAR(191),
                   'pruid' : DOUBLE,
                   'geo_name': VARCHAR(191),
                   'land_area' : DOUBLE,
                   'coordinates' : JSON
                  }

In [47]:
upload_df = pd.DataFrame(upload_sql)

upload_df["geo_level"] = upload_df['geo_level'].astype("string")
upload_df["dguid"] = upload_df['dguid'].astype("string")
upload_df["pruid"] = pd.to_numeric(upload_df["pruid"])
upload_df["coordinates"] = upload_df['coordinates'].astype("string")

In [48]:
upload_df.head()

,geo_level,dguid,pruid,geo_name,land_area,coordinates
0,province,2021A000210,10,Newfoundland and Labrador,3.581704e+05,"[[[[-56.8231629793897, 53.80003110535971], [-5..."
1,province,2021A000211,11,Prince Edward Island,5.681179e+03,"[[[[-61.986060886982244, 46.46285569495182], [..."
2,province,2021A000212,12,Nova Scotia,5.282471e+04,"[[[[-60.890609893286005, 45.576858598311695], ..."
3,province,2021A000213,13,New Brunswick,7.124850e+04,"[[[[-66.90309091045383, 44.95715030411355], [-..."
4,province,2021A000224,24,Quebec,1.298600e+06,"[[[[-69.23754572462461, 59.16254229713314], [-..."


In [50]:
upload_df.describe

<bound method NDFrame.describe of                  geo_level              dguid  pruid  \
0                 province        2021A000210     10   
1                 province        2021A000211     11   
2                 province        2021A000212     12   
3                 province        2021A000213     13   
4                 province        2021A000224     24   
...                    ...                ...    ...   
63394  dissemination areas  2021S051262080024     62   
63395  dissemination areas  2021S051262080025     62   
63396  dissemination areas  2021S051262080026     62   
63397  dissemination areas  2021S051262080027     62   
63398  dissemination areas  2021S051262080028     62   

                        geo_name     land_area  \
0      Newfoundland and Labrador  3.581704e+05   
1           Prince Edward Island  5.681179e+03   
2                    Nova Scotia  5.282471e+04   
3                  New Brunswick  7.124850e+04   
4                         Quebec  1.298600e

In [61]:
upload_df.dtypes

geo_level       string
dguid           string
pruid            int64
geo_name        object
land_area      float64
coordinates     string
dtype: object

In [62]:
with engine.begin() as connection:
    upload_df.to_sql('geo_areas',dtype=data_types_dict, con=connection, if_exists='replace')

In [65]:
with engine.begin() as connection:
    read_test = pd.read_sql("SELECT * FROM geo_areas WHERE geo_level = 'province' ", con=connection)
print(read_test)

    index geo_level        dguid  pruid                   geo_name  \
0       0  province  2021A000210   10.0  Newfoundland and Labrador   
1       1  province  2021A000211   11.0       Prince Edward Island   
2       2  province  2021A000212   12.0                Nova Scotia   
3       3  province  2021A000213   13.0              New Brunswick   
4       4  province  2021A000224   24.0                     Quebec   
5       5  province  2021A000235   35.0                    Ontario   
6       6  province  2021A000246   46.0                   Manitoba   
7       7  province  2021A000247   47.0               Saskatchewan   
8       8  province  2021A000248   48.0                    Alberta   
9       9  province  2021A000259   59.0           British Columbia   
10     10  province  2021A000260   60.0                      Yukon   
11     11  province  2021A000261   61.0      Northwest Territories   
12     12  province  2021A000262   62.0                    Nunavut   

       land_area   

In [63]:
with engine.begin() as connection:
    read_test_2 = pd.read_sql("SELECT * FROM geo_areas WHERE geo_level = 'dissemination areas'", con=connection)
print(read_test_2)

       index            geo_level              dguid  pruid  geo_name  \
0       5467  dissemination areas  2021S051210010165   10.0  10010165   
1       5468  dissemination areas  2021S051210010166   10.0  10010166   
2       5469  dissemination areas  2021S051210010167   10.0  10010167   
3       5470  dissemination areas  2021S051210010168   10.0  10010168   
4       5471  dissemination areas  2021S051210010169   10.0  10010169   
...      ...                  ...                ...    ...       ...   
57927  63394  dissemination areas  2021S051262080024   62.0  62080024   
57928  63395  dissemination areas  2021S051262080025   62.0  62080025   
57929  63396  dissemination areas  2021S051262080026   62.0  62080026   
57930  63397  dissemination areas  2021S051262080027   62.0  62080027   
57931  63398  dissemination areas  2021S051262080028   62.0  62080028   

       land_area                                        coordinates  
0         0.3817  "[[[[-52.77649809557013, 47.5292426

In [16]:
PROVINCE_DUMP = "/Users/boristsao/Documents/Projects/canada-vis/data-exploration/upload/provinces_new.json"
with open(PROVINCE_DUMP, 'w', encoding='utf-8') as f:
    json.dump(provinces, f, ensure_ascii=False, indent=4)

Work on Census Divisions

In [5]:
JSON_PATH = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/geojson/censusDivisions.json'
with open(JSON_PATH) as jsonfile:
    censusDivisions = json.load(jsonfile)

'CDUID' 'DGUID' 'CDNAME' 'LANDAREA' 'PRUID': '10'
'CDUID' 'DGUID' 'CDNAME' 'CDTYPE' 'LANDAREA': 802.5226, 'PRUID': '24'

In [6]:
for feature in censusDivisions["features"]:
    feature["properties"].pop('CDTYPE')
    feature["properties"]["geo_name"] = feature["properties"].pop('CDNAME')
    print(feature["properties"])

{'CDUID': '1001', 'DGUID': '2021A00031001', 'LANDAREA': 9104.5799, 'PRUID': '10', 'geo_name': 'Division No.  1'}
{'CDUID': '1002', 'DGUID': '2021A00031002', 'LANDAREA': 5915.5695, 'PRUID': '10', 'geo_name': 'Division No.  2'}
{'CDUID': '1003', 'DGUID': '2021A00031003', 'LANDAREA': 19272.1069, 'PRUID': '10', 'geo_name': 'Division No.  3'}
{'CDUID': '1004', 'DGUID': '2021A00031004', 'LANDAREA': 7019.9723, 'PRUID': '10', 'geo_name': 'Division No.  4'}
{'CDUID': '1005', 'DGUID': '2021A00031005', 'LANDAREA': 10293.7618, 'PRUID': '10', 'geo_name': 'Division No.  5'}
{'CDUID': '1006', 'DGUID': '2021A00031006', 'LANDAREA': 16043.3766, 'PRUID': '10', 'geo_name': 'Division No.  6'}
{'CDUID': '1007', 'DGUID': '2021A00031007', 'LANDAREA': 9598.3701, 'PRUID': '10', 'geo_name': 'Division No.  7'}
{'CDUID': '1008', 'DGUID': '2021A00031008', 'LANDAREA': 9234.9414, 'PRUID': '10', 'geo_name': 'Division No.  8'}
{'CDUID': '1009', 'DGUID': '2021A00031009', 'LANDAREA': 13209.4758, 'PRUID': '10', 'geo_name'

In [7]:
CENSUS_DIV_DUMP = "/Users/boristsao/Documents/Projects/canada-vis/data-exploration/upload/censusDivisions.json"
with open(CENSUS_DIV_DUMP, 'w', encoding='utf-8') as f:
    json.dump(censusDivisions, f, ensure_ascii=False, indent=4)

In [8]:
print(provinces["features"][0].keys())
print(provinces["features"][0]["properties"].keys())

dict_keys(['type', 'geometry', 'properties'])
dict_keys(['PRUID', 'DGUID', 'LANDAREA', 'geo_name', 'CDUID'])


In [9]:
print(censusDivisions["features"][0].keys())
print(censusDivisions["features"][0]["properties"].keys())

dict_keys(['type', 'geometry', 'properties'])
dict_keys(['CDUID', 'DGUID', 'LANDAREA', 'PRUID', 'geo_name'])


In [16]:
print(censusDivisions["features"][2]['geometry']['coordinates'])

[[[[-55.938787214801195, 47.70220400294832], [-56.044236493749864, 47.61464281181179], [-55.91263508509915, 47.59784080719939], [-55.90241399899436, 47.631333713277876], [-55.937233507680766, 47.65315709136787], [-55.938787214801195, 47.70220400294832]]], [[[-54.74714421698952, 48.11002919444765], [-54.91615167890936, 48.24537442769905], [-55.109432869930906, 48.246838688449515], [-55.24791566156148, 48.21521694249447], [-55.27854411593209, 48.268556852013035], [-55.442247976381125, 48.28249670457702], [-55.6253089334514, 48.25008203306865], [-55.89548614377431, 48.24697764016488], [-55.973320835742626, 48.436343753254675], [-56.133469778918, 48.53664501723045], [-56.283800872555496, 48.49228140832816], [-56.522285713239796, 48.525650094616175], [-56.67648997535755, 48.36734485521565], [-56.84789317730096, 48.30856793673976], [-57.01866632755163, 48.30398755121567], [-57.03331631006101, 48.36969981363618], [-57.17506647953183, 48.35784935401067], [-57.24659596314342, 48.47442260245579]